In [8]:
from tensorflow import keras
import os 
from IPython.display import SVG, display 
import tensorflow as tf 

In [9]:
output_directory = '/eos/user/a/atishelm/www/HHWWgg/DNN/HHWWyyDNN_Binary-10Epochs_BalanceYields/'
plots_dir = os.path.join(output_directory,'plots/')
modelFilename = "%s/model.h5"%(output_directory)
print("Loading model...")
model = keras.models.load_model(modelFilename)

Loading model...


W0222 21:35:40.328834 140677649540928 hdf5_format.py:211] Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.


In [11]:
tf.autograph.to_graph(model)
# svg_img = tf.autograph.to_graph(model).create(prog='dot', format='svg')
# SVG(svg_img)

ConversionError: converting <tensorflow.python.keras.engine.sequential.Sequential object at 0x7ff1ea4102e0>: ValueError: Cannot apply autograph to a function that doesn't expose a __code__ object. If this is a @tf.function, try passing f.python_function instead.

In [2]:
pip install --user shap 

     |████████████████████████████████| 358kB 6.6MB/s eta 0:00:01
     |████████████████████████████████| 81kB 10.2MB/s eta 0:00:01
  Stored in directory: /tmp/atishelm/.cache/pip/wheels/a8/fb/e4/88012be41842b9be62ae18d82d1b1e880daf8539d1fef1fa00
Successfully built shap
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [38]:
import numpy as np
import shap
import pandas
import os
import sklearn
import subprocess
from sklearn.metrics import roc_curve, roc_auc_score, auc, confusion_matrix
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import interp
from itertools import cycle

class plotter(object):

    def __init__(self, Website):
        self.separations_categories = []
        self.output_directory = ''
        #self.bin_edges_low_high = np.array([0.,0.0625,0.125,0.1875,0.25,0.3125,0.375,0.4375,0.5,0.5625,0.6125,0.6875,0.75,0.8125,0.875,0.9375,1.0])
        self.nbins = np.linspace(0.0,1.0,num=50)
        w, h = 4, 4
        self.yscores_train_categorised = [[0 for x in range(w)] for y in range(h)]
        self.yscores_test_categorised = [[0 for x in range(w)] for y in range(h)]
        self.yscores_train_non_categorised = [[0 for x in range(w)] for y in range(h)]
        self.yscores_test_non_categorised = [[0 for x in range(w)] for y in range(h)]
        self.plots_directory = ''
        self.Website = Website 
        pass

    def save_plots(self, dir='plots/', filename=''):
        self.check_dir(dir)
        filepath = os.path.join(dir,filename)
        self.fig.savefig(filepath)
        return self.fig    
    
    def check_dir(self, dir):
        if not os.path.exists(dir):
            os.makedirs(dir)
            if(self.Website != ""): os.system("cp %s/../index.php %s"%(dir,dir))    
    
    def binary_overfitting(self, estimator, Y_train, Y_test, result_probs, result_probs_test, plots_dir, train_weights, test_weights, nbins=50):

        model = estimator
        data_type = type(model)

        #Arrays to store all results
        y_scores_train_signal_sample = []
        y_scores_train_bckg_sample = []
        y_scores_test_signal_sample = []
        y_scores_test_bckg_sample = []
        for i in range(0,len(result_probs)-1):
            train_event_weight = train_weights[i]
            if Y_train[i] == 1:
                y_scores_train_signal_sample.append(result_probs[i])
            if Y_train[i] == 0:
                y_scores_train_bckg_sample.append(result_probs[i])
        for i in range(0,len(result_probs_test)-1):
            test_event_weight = test_weights[i]
            if Y_test[i] == 1:
                y_scores_test_signal_sample.append(result_probs_test[i])
            if Y_test[i] == 0:
                y_scores_test_bckg_sample.append(result_probs_test[i])

        # Create 2D lists (dimension 2x2) to hold max DNN discriminator values for each sample. One for train data, one for test data.
        yscores_train_binary=[]
        yscores_test_binary=[]
        yscores_train_binary.append([y_scores_train_signal_sample, y_scores_train_bckg_sample])
        yscores_test_binary.append([y_scores_test_signal_sample, y_scores_test_bckg_sample])

        counter =0
        separations_all = []
        for y_scores_train_nonCat,y_scores_test_nonCat in zip(yscores_train_binary,yscores_test_binary):
            colours = ['r','steelblue']
            plot_title = 'Binary'
            plot_info = [colours,data_type,plots_dir,plot_title]
            separations_all.append(self.draw_binary_overfitting_plot(y_scores_train_nonCat,y_scores_test_nonCat,plot_info,test_weights))
            counter = counter+1

        return
    def plot_dot(self, title, x, shap_values, column_headers):
        plt.figure()
        if x is None:
          print('<plotter> No x defined. Leaving class function')
          return
        shap.summary_plot(shap_values[0], features=x, feature_names=column_headers, show=False, max_display=10)
        plt.gca().set_title(title)
        plt.tight_layout()
        plt.savefig("{}/plots/{}.png".format(self.output_directory, title), bbox_inches='tight')
        plt.savefig("{}/plots/{}.pdf".format(self.output_directory, title), bbox_inches='tight')

    def plot_dot_bar(self, title, x, shap_values, column_headers):
        plt.figure()
        if x is None:
            print('<plotter> No x defined. Leaving class function')
            return
        shap.summary_plot(shap_values[0], features=x, feature_names=column_headers, show=False,plot_type='bar',max_display=10)
        plt.gca().set_title(title)
        plt.tight_layout()
        plt.savefig("{}/plots/{}.png".format(self.output_directory,title), bbox_inches='tight')
        plt.savefig("{}/plots/{}.pdf".format(self.output_directory,title), bbox_inches='tight')

    def plot_dot_bar_all(self , title, x, shap_values, column_headers):
        plt.figure()
        if x is None:
            print('<plotter> No x defined. Leaving class function')
            return
        shap.summary_plot(shap_values[0], features=x, feature_names=column_headers, show=False,plot_type='bar',max_display=len(column_headers))
        plt.gca().set_title(title)
        plt.tight_layout()
        plt.savefig("{}/plots/{}.png".format(self.output_directory,title), bbox_inches='tight')
        plt.savefig("{}/plots/{}.pdf".format(self.output_directory,title), bbox_inches='tight')

    def GetSeparation(self, hist_sig, hist_bckg):


        minima = np.minimum(hist_sig, hist_bckg)
        intersection = np.true_divide(np.sum(minima), np.sum(hist_bckg))
        return intersection

        '''
        # compute "separation" defined as
        # <s2> = (1/2) Int_-oo..+oo { (S(x) - B(x))^2/(S(x) + B(x)) dx }
        separation = 0;
        # sanity check: signal and background histograms must have same number of bins and same limits
        if len(hist_sig) != len(hist_bckg):
            print 'Number of bins different for sig. and bckg'

        nBins = len(hist_sig)
        nS = np.sum(hist_sig)
        nB = np.sum(hist_bckg)

        if nS == 0:
            print 'WARNING: no signal'
        if nB == 0:
            print 'WARNING: no bckg'

        for i in range(1,nBins):
            # No need to norm as already done?
            sig_bin_norm = hist_sig[i]/nS
            #sig_bin_norm = hist_sig[i]
            bckg_bin_norm = hist_bckg[i]/nB
            #bckg_bin_norm = hist_bckg[i]
            # Separation:
            if(sig_bin_norm+bckg_bin_norm > 0):
                separation += 0.5 * ((sig_bin_norm - bckg_bin_norm) * (sig_bin_norm - bckg_bin_norm)) / (sig_bin_norm + bckg_bin_norm)
        #separation *= dX
        return separation'''         
        
    def draw_binary_overfitting_plot(self, y_scores_train, y_scores_test, plot_info, test_weights):
        colours = plot_info[0]
        data_type = plot_info[1]
        plots_dir = plot_info[2]
        plot_title = plot_info[3]
        name = filter(str.isalnum, str(data_type).split(".")[-1])
        self.fig, self.ax = plt.subplots(figsize=(8,6))
        self.ax.grid(which='major', linestyle='-', linewidth='0.2', color='gray')
        self.ax.set_facecolor('white')

        bin_edges_low_high = np.array([0.,0.0625,0.125,0.1875,0.25,0.3125,0.375,0.4375,0.5,0.5625,0.6125,0.675,0.7375,0.8,0.8625,0.9375,1.0])

        index=0
        for index in range(0,len(y_scores_train)):
            train_bin_errors = np.zeros(len(bin_edges_low_high)-1)
            test_bin_errors = np.zeros(len(bin_edges_low_high)-1)

            y_train = y_scores_train[index]
            y_test = y_scores_test[index]
            colour = colours[index]
            width = np.diff(bin_edges_low_high)
            if index==0:
                print('<plotter> Overfitting plot: Signal')
                label='signal'
            if index==1:
                label='bckg'
                print('<plotter> Overfitting plot: Background')

            # Setup training histograms
            histo_train_, bin_edges = np.histogram(y_train, bins=bin_edges_low_high)
            dx_scale_train =(bin_edges[len(bin_edges)-1] - bin_edges[0]) / (len(bin_edges)-1)
            bincenters = 0.5*(bin_edges[1:]+bin_edges[:-1])

            bin_errors_sumw2 = 0

            # Scale training histograms to: (hist / sum of histogram entries) / (range / number of bins)
            # so like scaling hist relative to its integral.
            # then scaling the result relative to its avergae bin width.
            histo_train_ = (histo_train_ / np.sum(histo_train_, dtype=np.float32)) / dx_scale_train
            plt.bar(bincenters, histo_train_, width=width, color=colour, edgecolor=colour, alpha=0.5, label=label+' training')

            if index == 0:
                histo_train_sig = histo_train_
            if index == 1:
                histo_train_bckg = histo_train_

            histo_test_, bin_edges = np.histogram(y_test, bins=bin_edges_low_high)
            dx_scale_test =(bin_edges[len(bin_edges)-1] - bin_edges[0]) / (len(bin_edges)-1)
            bincenters = 0.5*(bin_edges[1:]+bin_edges[:-1])

            # Protection against low stats in validation dataset.
            if np.sum(histo_test_, dtype=np.float32) <= 0 :
                histo_test_ = histo_test_
                err = 0
                plt.errorbar(bincenters, histo_test_, yerr=err, fmt='o', c=colour, label=label+' testing')
                if index == 0:
                    histo_test_sig = histo_test_
                if index == 1:
                    histo_test_bckg = histo_test_
            else:
                # Currently just taking the sqrt of the bin entry
                # Correct way:
                # Errors calculated using error propogation and the histograms intrinsic poissonian statistics
                # err(sum weights)^2 == variance on the sum of weights = sum of the variance of each weight =  sum{var(w_i)} [i=1,2,N]
                # Varianceof weight i is determined by the statistical fluctuation of the number of events considered.
                # var(w_i) = var(w_i * 1 event) = w_i^2 * var(1 event) = w_i^2
                # err(sum weights) = sqrt( sum{var(w_i)}[i=1,2,N] )
                #                  = sqrt( sum{w_i^2}[i=1,2,N] )
                bin_errors_sumw2 = 0
                for yval_index in range(0,len(y_scores_test[0])):
                    for bin_index in range(0,len(bin_edges_low_high)-1):
                        bin_low_edge = bin_edges_low_high[bin_index]
                        bin_high_edge = bin_edges_low_high[bin_index+1]
                        if y_scores_test[0][yval_index] > bin_low_edge and y_scores_test[0][yval_index] < bin_high_edge:
                            test_bin_errors[[bin_index]] += 1#test_weights[yval_index]**2
                # Take square root of sum of bins.
                test_bin_errors = (np.sqrt(test_bin_errors)/np.sum(histo_test_, dtype=np.float32)) / dx_scale_test
                #test_bin_errors = np.sqrt(( test_bin_errors/np.sum(histo_test_, dtype=np.float32) ) / dx_scale_test )
                histo_test_ = ( histo_test_ / np.sum(histo_test_, dtype=np.float32) ) / dx_scale_test
                #err = np.sqrt(histo_test_/np.sum(histo_test_, dtype=np.float32))
                #err = np.sqrt(histo_test_)

                plt.errorbar(bincenters, histo_test_, yerr=test_bin_errors, fmt='o', c=colour, label=label+' testing')
                if index == 0:
                    histo_test_sig = histo_test_
                if index == 1:
                    histo_test_bckg = histo_test_


        train_SvsBSep = "{0:.5g}".format(self.GetSeparation(histo_train_sig,histo_train_bckg))
        test_SvsBSep = "{0:.5g}".format(self.GetSeparation(histo_test_sig,histo_test_bckg))

        S_v_B_train_sep = 'SvsB train Sep.: %s' % ( train_SvsBSep )
        self.ax.annotate(S_v_B_train_sep,  xy=(0.7, 2.5), xytext=(0.7, 2.5), fontsize=9)
        S_v_B_test_sep = 'SvsB test Sep.: %s' % ( test_SvsBSep )
        self.ax.annotate(S_v_B_test_sep,  xy=(0.7, 1.75), xytext=(0.7, 1.5), fontsize=9)

        separations_forTable = r'''%s & \textbackslash ''' % (S_v_B_test_sep)

        title_ = '%s output node' % (plot_title)
        plt.title(title_)
        label_name = 'Output Score'
        plt.xlabel(label_name)
        plt.ylabel('(1/N)dN/dX')

        leg = plt.legend(loc='best', frameon=False, fancybox=False, fontsize=9)
        leg.get_frame().set_edgecolor('w')
        frame = leg.get_frame()
        frame.set_facecolor('White')

        overfitting_plot_file_name = 'overfitting_plot_BinaryClassifier_%s.pdf' % (plot_title)
        print('Saving : %s%s' % (plots_dir, overfitting_plot_file_name))
        self.save_plots(dir=plots_dir, filename=overfitting_plot_file_name)
        
#         overfitting_plot_file_name = 'overfitting_plot_BinaryClassifier_%s.pdf' % (plot_title)
#         print('Saving : %s%s' % (plots_dir, overfitting_plot_file_name))
#         self.save_plots(dir=plots_dir, filename=overfitting_plot_file_name)        
        return separations_forTable     
       

In [1]:
# from tensorflow import keras 
import argparse
import pickle 
import os 
import pandas as pd 
import h5py 
from keras.utils import plot_model

Using TensorFlow backend.


In [3]:
from tensorflow import keras
from tensorflow.keras.utils import plot_model

In [4]:
# output_directory = '/eos/user/a/atishelm/www/DNN/HHWWyyDNN_200Epochs-MultiClass-2Classes_BalanceYields/'
output_directory = '/eos/user/a/atishelm/www/HHWWgg/DNN/HHWWyyDNN_Binary-10Epochs_BalanceYields/'
plots_dir = os.path.join(output_directory,'plots/')
modelFilename = "%s/model.h5"%(output_directory)
print("Loading model...")
model = keras.models.load_model(modelFilename)

objectsToLoad = ["X_train", "X_test", "Y_test","Y_train","train_weights","test_weights"]
for objToLoad in objectsToLoad:
    print("Loading %s..."%(objToLoad))
    executeLine = "%s = pickle.load( open( '%s/%s.p', 'rb' ), encoding='latin1' )"%(objToLoad, output_directory, objToLoad) ##-- encoding='latin1' to avoid python 2 / 3 numpy array incompatibility: https://stackoverflow.com/questions/11305790/pickle-incompatibility-of-numpy-arrays-between-python-2-and-3
    print(executeLine)
    exec(executeLine)

result_probs = model.predict(np.array(X_train))
result_probs_test = model.predict(np.array(X_test))
# train_weights = abs(traindataset['weight'].values)*abs(traindataset['weight_NLO_SM'].values)
# test_weights = abs(valdataset['weight'].values)*abs(valdataset['weight_NLO_SM'].values)

Loading model...


W0222 08:54:19.803083 140074485446464 hdf5_format.py:211] Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.


Loading X_train...
X_train = pickle.load( open( '/eos/user/a/atishelm/www/HHWWgg/DNN/HHWWyyDNN_Binary-10Epochs_BalanceYields//X_train.p', 'rb' ), encoding='latin1' )
Loading X_test...
X_test = pickle.load( open( '/eos/user/a/atishelm/www/HHWWgg/DNN/HHWWyyDNN_Binary-10Epochs_BalanceYields//X_test.p', 'rb' ), encoding='latin1' )
Loading Y_test...
Y_test = pickle.load( open( '/eos/user/a/atishelm/www/HHWWgg/DNN/HHWWyyDNN_Binary-10Epochs_BalanceYields//Y_test.p', 'rb' ), encoding='latin1' )
Loading Y_train...
Y_train = pickle.load( open( '/eos/user/a/atishelm/www/HHWWgg/DNN/HHWWyyDNN_Binary-10Epochs_BalanceYields//Y_train.p', 'rb' ), encoding='latin1' )
Loading train_weights...
train_weights = pickle.load( open( '/eos/user/a/atishelm/www/HHWWgg/DNN/HHWWyyDNN_Binary-10Epochs_BalanceYields//train_weights.p', 'rb' ), encoding='latin1' )
Loading test_weights...
test_weights = pickle.load( open( '/eos/user/a/atishelm/www/HHWWgg/DNN/HHWWyyDNN_Binary-10Epochs_BalanceYields//test_weights.p', 'rb' 

NameError: name 'np' is not defined

In [5]:
output_directory = '/eos/user/a/atishelm/www/HHWWgg/DNN/HHWWyyDNN_Binary-10Epochs_BalanceYields/'
plots_dir = os.path.join(output_directory,'plots/')
modelFilename = "%s/model.h5"%(output_directory)
print("Loading model...")
model = keras.models.load_model(modelFilename)

model_schematic_name = os.path.join(output_directory,'model_schematic.pdf')
plot_model(model, to_file=model_schematic_name, show_shapes=True, show_layer_names=True) 

Loading model...


W0222 08:54:27.416645 140074485446464 hdf5_format.py:211] Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.


In [39]:
Website = '/eos/user/a/atishelm/www/HHWWgg/DNN/'
Plotter = plotter(Website)

In [44]:
# Make overfitting plots of output nodes



# Plotter.binary_overfitting(model, Y_train, Y_test, result_probs, result_probs_test, plots_dir, train_weights, test_weights)
e = shap.DeepExplainer(model, X_train[:400, ])
shap_values = e.shap_values(X_test[:400, ])
# Plotter.plot_dot(title="DeepExplainer_sigmoid_y0", x=X_test[:400, ], shap_values=shap_values, column_headers=column_headers)
# Plotter.plot_dot_bar(title="DeepExplainer_Bar_sigmoid_y0", x=X_test[:400,], shap_values=shap_values, column_headers=column_headers)

# e = shap.GradientExplainer(model, X_train[:100, ])
# shap_values = e.shap_values(X_test[:100, ])
# Plotter.plot_dot(title="GradientExplainer_sigmoid_y0", x=X_test[:100, ], shap_values=shap_values, column_headers=column_headers)
# e = shap.KernelExplainer(model.predict, X_train[:100, ])
# shap_values = e.shap_values(X_test[:100, ])
# Plotter.plot_dot(title="KernelExplainer_sigmoid_y0", x=X_test[:100, ],shap_values=shap_values, column_headers=column_headers)
# Plotter.plot_dot_bar(title="KernelExplainer_Bar_sigmoid_y0", x=X_test[:100,], shap_values=shap_values, column_headers=column_headers)
# Plotter.plot_dot_bar_all(title="KernelExplainer_bar_All_Var_sigmoid_y0", x=X_test[:100,], shap_values=shap_values, column_headers=column_headers)

StagingError: in user code:

    /eos/user/a/atishelm/.local/lib/python3.8/site-packages/shap/explainers/_deep/deep_tf.py:248 grad_graph  *
        x_grad = tape.gradient(out, shap_rAnD)
    /cvmfs/sft.cern.ch/lcg/views/LCG_99/x86_64-centos7-gcc8-opt/lib/python3.8/site-packages/tensorflow/python/eager/backprop.py:1067 gradient  **
        flat_grad = imperative_grad.imperative_grad(
    /cvmfs/sft.cern.ch/lcg/views/LCG_99/x86_64-centos7-gcc8-opt/lib/python3.8/site-packages/tensorflow/python/eager/imperative_grad.py:71 imperative_grad
        return pywrap_tfe.TFE_Py_TapeGradient(
    /cvmfs/sft.cern.ch/lcg/views/LCG_99/x86_64-centos7-gcc8-opt/lib/python3.8/site-packages/tensorflow/python/eager/backprop.py:151 _gradient_function
        grad_fn = ops._gradient_registry.lookup(op_name)  # pylint: disable=protected-access
    /cvmfs/sft.cern.ch/lcg/views/LCG_99/x86_64-centos7-gcc8-opt/lib/python3.8/site-packages/tensorflow/python/framework/registry.py:96 lookup
        raise LookupError(
